In [132]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_ollama import ChatOllama
from langchain.prompts import PromptTemplate
from IPython.display import Markdown, display
from langchain_core.output_parsers import StrOutputParser
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.memory import SimpleMemory
from datetime import datetime

In [133]:
llm=ChatOllama(model="llama3")

In [134]:
pdf_file_path = ("/Users/rakeshbidhar/Downloads/Generative-AI-and-LLMs-for-Dummies.pdf")

In [135]:
# from https://python.langchain.com/docs/how_to/document_loader_pdf/
loader = PyPDFLoader(pdf_file_path)
pages = []
async for page in loader.alazy_load():
    pages.append(page)

#print(f"{pages[0].metadata}\n")
#print(pages[6].page_content)

In [136]:
chapters = pages[4].page_content + pages[5].page_content

In [137]:
# We want to use PromptTemplate() from langchain. 
# Lets define a template for extracting chapters and its starting page and ending page
# Next we will use these page numbers to summarize by Chapters.

chapters_template = """Can you list down the chapter names
and its start page number and end page number. Page numbers are followed by the dots. 
Use this {chapters}. """

In [138]:
chapters_prompt = PromptTemplate(input_variables=["chapters"], output_variables=["chapters_page_indicators"], template=chapters_template)

In [149]:
chapters_chain: LLMChain = LLMChain(
    llm=llm, output_key="chapters_page_indicators", prompt=chapters_prompt)

In [155]:
summary_template = """You are an expert in summarizing text by Chapters. 
A chapter starts at a certain page and ends at certain page. Its mentioned in {chapters_page_indicators}.
Using this, summarize the chapter contents in these {pages} for all the chapters.

The Summary should have structure as below.
- **Chapter 1 **: Chapter Name.
- **Summary**: Summary in 1000 words 

- **Chapter 2 **: Chapter Name.
- **Summary**: Summary in 1000 words 

- **Chapter 3 **: Chapter Name.
- **Summary**: Summary in 1000 words 

- **Chapter 4 **: Chapter Name.
- **Summary**: Summary in 1000 words 

"""

In [156]:
summary_by_chapter_prompt = PromptTemplate(input_variables=["chapters_page_indicators", "pages"]
                                           , template=summary_template)

In [157]:
summary_by_chapter_chain: LLMChain = LLMChain(
    llm=llm, output_key="final_output", prompt=summary_by_chapter_prompt)

In [158]:
ss_chain: SequentialChain = SequentialChain(
    memory=SimpleMemory(memories={
                        "time_created_and_verified": str(datetime.utcnow()), "verified_by_human": "False", "pages": pages}),
    chains=[chapters_chain, summary_by_chapter_chain],
    # multiple variables
    input_variables=["chapters"],
    output_variables=["final_output"],
    verbose=True)

# running chain
review = ss_chain.run(chapters=chapters)
print(review)

/var/folders/9q/k8ncmmvj1q57w_pmw02pmkl40000gn/T/ipykernel_10387/1836860453.py:3: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  "time_created_and_verified": str(datetime.utcnow()), "verified_by_human": "False", "pages": pages}),




> Entering new SequentialChain chain...

> Finished chain.
Here is the summary of the text by chapters:

**CHAPTER 1:**

The chapter discusses the importance of generative AI and its potential to revolutionize industries. It highlights the need for enterprises to develop a deep understanding of AI and its applications to stay ahead in the market. The chapter also emphasizes the importance of data governance and privacy, as well as the need for a strong culture of collaboration within organizations.

**Summary:** Generative AI has the potential to transform businesses by enabling them to automate tasks, make predictions, and generate new ideas. However, enterprises must first develop a deep understanding of AI and its applications to stay ahead in the market. This includes implementing data governance and privacy measures to protect sensitive information and fostering a culture of collaboration within organizations.

**CHAPTER 2:**

This chapter focuses on the concept of language mode

In [68]:
llm=ChatOllama(model="llama3")
response = llm.invoke(prompt.format(chapters=chapters))
print(response.content)

Here are the chapter names, start page numbers, and end page numbers:

**CHAPTER 1: Introducing Gen AI and the Role of Data**
Start Page: 3
End Page: 10

**CHAPTER 2: Understanding Large Language Models**
Start Page: 11
End Page: 18

**CHAPTER 3: LLM App Project Lifecycle**
Start Page: 19
End Page: 25

**CHAPTER 4: Bringing LLM Apps into Production**
Start Page: 29
End Page: 34

**CHAPTER 5: Reviewing Security and Ethical Considerations**
Start Page: 37
End Page: 42

**CHAPTER 6: Five Steps to Generative AI**
Start Page: 43
End Page: 44


In [59]:
display(Markdown(response.content))

Here is the list of chapter names with their start page numbers and end page numbers:

**CHAPTER 1: Introducing Gen AI and the Role of Data**
Start: 3
End: 10

**CHAPTER 2: Understanding Large Language Models**
Start: 11
End: 18

**CHAPTER 3: LLM App Project Lifecycle**
Start: 19
End: 25

**CHAPTER 4: Bringing LLM Apps into Production**
Start: 29
End: 34

**CHAPTER 5: Reviewing Security and Ethical Considerations**
Start: 37
End: 42

**CHAPTER 6: Five Steps to Generative AI**
Start: 43
End: 44

In [16]:
# make api call
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": prompt},
    ], 
    temperature = 0.5
)

# extract response
summary = response.choices[0].message.content
print(summary)

# System Role: Generative AI and LLM Tutor
You are an expert in Generative AI and LLMs. Can you summarize the [Document(metadata={'source': '/Users/rakeshbidhar/Downloads/Generative-AI-and-LLMs-for-Dummies.pdf', 'page': 0, 'page_label': 'C1'}, page_content=''), Document(metadata={'source': '/Users/rakeshbidhar/Downloads/Generative-AI-and-LLMs-for-Dummies.pdf', 'page': 1, 'page_label': 'C2'}, page_content='These materials are © 2024 John Wiley & Sons, Inc. Any dissemination, distribution, or unauthorized use is strictly prohibited.'), Document(metadata={'source': '/Users/rakeshbidhar/Downloads/Generative-AI-and-LLMs-for-Dummies.pdf', 'page': 2, 'page_label': 'i'}, page_content='These materials are © 2024 John Wiley & Sons, Inc. Any dissemination, distribution, or unauthorized use is strictly prohibited.\nGenerative AI \nand LLMs\nSnowflake Special Edition\nby David Baum'), Document(metadata={'source': '/Users/rakeshbidhar/Downloads/Generative-AI-and-LLMs-for-Dummies.pdf', 'page': 3, 'pa